# Competição de Machine Learning Kaggle Titanic
by Aline Assunção

Site utilizados para a construção da solução: https://www.kaggle.com/c/titanic,  http://mundoia.com.br/tutorial/conheca-o-kaggle-e-participe-da-sua-primeira-competicao-de-machine-learning/ e https://guidefreitas.github.io/ia/machinelearning/2017/04/10/Kaggle-Titanic.html

### Descrição da Competição

O naufrágio do RMS Titanic é um dos mais infames naufrágios da história. Em 15 de abril de 1912, durante sua viagem inaugural, o Titanic afundou depois de colidir com um iceberg, matando 1502 de 2224 passageiros e tripulantes. Essa tragédia sensacional chocou a comunidade internacional e levou a melhores normas de segurança para os navios.

Uma das razões pelas quais o naufrágio causou tamanha perda de vidas foi que não havia botes salva-vidas suficientes para os passageiros e a tripulação. Embora houvesse algum elemento de sorte envolvido na sobrevivência do naufrágio, alguns grupos de pessoas tinham maior probabilidade de sobreviver do que outros, como mulheres, crianças e a classe alta.

Neste desafio, é solicitado que concluamos a análise de quais tipos de pessoas provavelmente sobreviveriam. Aplicando as ferramentas de aprendizado de máquina para prever quais passageiros sobreviveram à tragédia.

Praticar Habilidades
- Classificação binária
- Noções básicas de Python

### Objetivo

O trabalho é prever se um passageiro sobreviveu ao naufrágio do Titanic ou não.
Para cada PassengerId no conjunto de testes, deve-se prever um valor 0 ou 1 para a variável Survived.


### Métrica

A pontuação é a porcentagem de passageiros que você prevê corretamente. Isso é conhecido simplesmente como "acurácia".


### Formato de Arquivo de Submissão

Enviar um arquivo csv com exatamente 418 entradas mais uma linha de cabeçalho. O envio mostrará um erro se tiver colunas extras (além de PassengerId e Survived) ou linhas.

O arquivo deve ter exatamente 2 colunas:
- PassengerId (classificado em qualquer ordem)
- Survived (contém suas previsões binárias: 1 para sobreviventes, 0 para mortos)

### Descrição dos Dados

Temos dois datasets uma para treinar o modelo e outro para o envio do arquivo que o Kaggle espera.
- training set (train.csv)
- test set (test.csv)

### Dicionário dos dados

PassengerId: Número de identificação do passageiro;

Survived: Indica se o passageiro sobreviveu ao desastre. É atribuído o valor de 0 para aqueles que não sobreviveram, e 1 para quem sobreviveu;

Pclass: Classe na qual o passageiro viajou. É informado 1 para primeira classe; 2 para segunda; e 3 para terceira;

Name: Nome do passageiro;

Sex: Sexo do passageiro;

Age: Idade do passageiro em anos;

SibSp: Quantidade de irmãos e cônjuges a bordo ;

Parch: Quantidade de pais e filhos a bordo;

Ticket: Número da passagem;

Fare: Preço da passagem;

Cabin: Número da cabine do passageiro;

Embarked: Indica o porto no qual o passageiro embarcou. Há apenas três valores possíveis: Cherbourg, Queenstown e Southampton, indicados pelas letras “C”, “Q” e “S”, respectivamente.

In [1]:
# Primeiro importamos as bibliotecas que precisamos. 
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [5]:
# lendo os conjuntos de teste e treino - read_csv do Pandas
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
full = train.append( test , ignore_index = True )
titanic = full[ :891 ]

In [6]:
sex = pd.Series( np.where( full.Sex == 'male' , 1 , 0 ) , name = 'Sex' )

In [7]:
embarked = pd.get_dummies( full.Embarked , prefix='Embarked' )

In [8]:
pclass = pd.get_dummies( full.Pclass , prefix='Pclass' )

In [9]:
imputed = pd.DataFrame()
imputed[ 'Age' ] = full.Age.fillna( full.Age.mean() )
imputed[ 'Fare' ] = full.Fare.fillna( full.Fare.mean() )

In [10]:
cabin = pd.DataFrame()

#Substitui os dados de cabines faltantes com U (Unknown)
cabin[ 'Cabin' ] = full.Cabin.fillna( 'U' )

# mapping each Cabin value with the cabin letter
cabin[ 'Cabin' ] = cabin[ 'Cabin' ].map( lambda c : c[0] )
cabin = pd.get_dummies( cabin['Cabin'] , prefix = 'Cabin' )

In [11]:
full_X = pd.concat( [ imputed , embarked , pclass , sex, cabin ] , axis=1 )
full_X.tail()

,Age,Fare,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3,Sex,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U
1304,29.881138,8.0500,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1
1305,39.000000,108.9000,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0
1306,38.500000,7.2500,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1
1307,29.881138,8.0500,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1
1308,29.881138,22.3583,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1


In [12]:
from sklearn.model_selection import train_test_split

full_X_norm = full_X.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
full_X['Age'] = full_X_norm['Age']
full_X['Fare'] = full_X_norm['Fare']

In [15]:
train_valid_X = full_X[ 0:891 ]
train_valid_y = titanic.Survived
test_X = full_X[ 891: ]
train_X , valid_X , train_y , valid_y = train_test_split( train_valid_X , train_valid_y , train_size = 0.7 )

In [20]:
# SVM (Support Vector Machine)
import warnings
warnings.filterwarnings('ignore')
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

In [21]:
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-2, 1e-3, 1e-4, 1e-5],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

scores = ['precision', 'recall']

for score in scores:
    model = GridSearchCV(SVC(C=1), tuned_parameters, cv=5,
                           scoring='%s_macro' % score)
    model.fit(train_X , train_y)

    # Score the model
    print (model.score( train_X , train_y ) , model.score( valid_X , valid_y ))

    print("Best parameters set found on development set:")
    print()
    print(model.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = model.cv_results_['mean_test_score']
    stds = model.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, model.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = valid_y, model.predict(valid_X)
    print(classification_report(y_true, y_pred))
    print()

0.8206420867820416 0.8493324649951156
Best parameters set found on development set:

{'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}

Grid scores on development set:

0.754 (+/-0.050) for {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}
0.306 (+/-0.002) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.306 (+/-0.002) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.306 (+/-0.002) for {'C': 1, 'gamma': 1e-05, 'kernel': 'rbf'}
0.747 (+/-0.022) for {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
0.750 (+/-0.034) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.306 (+/-0.002) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.306 (+/-0.002) for {'C': 10, 'gamma': 1e-05, 'kernel': 'rbf'}
0.741 (+/-0.053) for {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
0.747 (+/-0.022) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.749 (+/-0.031) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.306 (+/-0.002) for {'C': 100, 'gamma': 1e-05, 'kernel': 'rbf'}
0.791 (+/-0.042) for {'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}
0.73

In [22]:
t_X = test_X.as_matrix()

test_Y = model.predict( t_X )
test_Y = test_Y.flatten().round().astype(int)
passenger_id = full[891:].PassengerId
test = pd.DataFrame( { 'PassengerId': passenger_id , 'Survived': test_Y } )
test.shape
test.head()
test.to_csv( 'result/submission_SVM.csv' , index = False )


In [23]:
# Multi-layer Perception - MLP (Scikit Learn)
from sklearn.neural_network import MLPClassifier

In [24]:
model = MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(6, 2), random_state=1)
model.fit( train_X , train_y )

# Score the model
print ("Train: ", model.score( train_X , train_y ))
print ("Test:", model.score( valid_X , valid_y ))

Train:  0.7784911717495987
Test: 0.8171641791044776


In [25]:
t_X = test_X.as_matrix()

test_Y = model.predict( t_X )
test_Y = test_Y.flatten().round().astype(int)
passenger_id = full[891:].PassengerId
test = pd.DataFrame( { 'PassengerId': passenger_id , 'Survived': test_Y } )
test.shape
test.head()
test.to_csv( 'result/submission_MLP.csv' , index = False )

In [26]:
from sklearn import tree
model = tree.DecisionTreeClassifier()
model.fit(train_X , train_y)
print ("Train: ", model.score( train_X , train_y ))
print ("Test:", model.score( valid_X , valid_y ))


Train:  0.985553772070626
Test: 0.8059701492537313


In [28]:
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(n_estimators=50)
model.fit(train_X , train_y)
print ("Train: ", model.score( train_X , train_y ))
print ("Test:", model.score( valid_X , valid_y ))


Train:  0.8202247191011236
Test: 0.8694029850746269


In [30]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
model.fit(train_X , train_y)
print ("Train: ", model.score( train_X , train_y ))
print ("Test:", model.score( valid_X , valid_y ))


Train:  0.8426966292134831
Test: 0.8619402985074627


In [31]:
t_X = test_X.as_matrix()

test_Y = model.predict( t_X )
test_Y = test_Y.flatten().round().astype(int)
passenger_id = full[891:].PassengerId
test = pd.DataFrame( { 'PassengerId': passenger_id , 'Survived': test_Y } )
test.shape
test.head()
test.to_csv( 'result/submission_DT3.csv' , index = False )

In [37]:
# Gaussian Naive Bayes
!pip install keras


In [39]:
from keras.utils.np_utils import to_categorical

t_X = train_X.as_matrix()
t_y = train_y.as_matrix()
#t_y = to_categorical(t_y)

v_X = valid_X.as_matrix()
v_y = valid_y.as_matrix()
#v_y = to_categorical(v_y)

test_X_m = test_X.as_matrix()
print(test_X_m)

[[ 5.78587290e-02 -4.97068668e-02  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  1.00000000e+00]
 [ 2.14441467e-01 -5.13253574e-02  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  1.00000000e+00]
 [ 4.02340753e-01 -4.60797067e-02  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  1.00000000e+00]
 ...
 [ 1.07965205e-01 -5.08373899e-02  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  1.00000000e+00]
 [ 3.56027927e-16 -4.92758939e-02  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  1.00000000e+00]
 [ 3.56027927e-16 -2.13479522e-02  1.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  1.00000000e+00]]


In [40]:
t_X = test_X.as_matrix()

test_Y = model.predict( t_X )
test_Y = test_Y.flatten().round().astype(int)
passenger_id = full[891:].PassengerId
test = pd.DataFrame( { 'PassengerId': passenger_id , 'Survived': test_Y } )
test.shape
test.head()
test.to_csv( 'result/submission_GNB.csv' , index = False )